In [1]:
import numpy as np
import pandas as pd
from urllib.parse import urlencode,urlparse,parse_qsl
from scr.Google import Create_Service
from scr.settings import *
from googleapiclient.http import MediaIoBaseDownload
import datetime as dt
from scr.sheet_utils import get_sheets
from PIL import Image
import io

In [2]:
service=Create_Service(CLIENT_SECRET_FILE,API_NAME,API_VERSION,TOKEN_PATH_DRIVE,SCOPES)

C:\Dropbox\Dropbox\PTK Nepal\rojan\Ptk\ptk-data\creds\client_secret.json-drive-v3-(['https://www.googleapis.com/auth/drive.file', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file', 'https://www.googleapis.com/auth/drive.metadata'],)
['https://www.googleapis.com/auth/drive.file', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file', 'https://www.googleapis.com/auth/drive.metadata']
refreshing #######
drive service created successfully


In [4]:
df_storage=get_sheets(
    sheet_filename='Storage_',
    work_sheet_name='Sheet1',
    TITLE_ROW= 1,
    TOKEN_PATH=TOKEN_PATH_SHEETS
)

In [27]:
def get_pic_id(x):
    try:
        id_name=x.split("=")[1]
        return id_name
    except (IndexError, AttributeError):
        return pd.NA

def get_fn(x):
    fn=x.split(' ')[0]
    return fn
def get_ln(x):
    ln=x.split(' ')[-1]
    return ln

def get_age(dob):
    try:
        birth_date=dt.datetime.strptime(dob,'%d/%m/%y')
        today=dt.datetime.today()
        age_year=((today-birth_date).days)/365
        return age_year
    except ValueError:
        return np.NaN

In [36]:
filt=df_storage['data-Name']=='' ## remove
df_storage=df_storage.loc[~filt]
df=pd.DataFrame()

In [37]:
df=df.assign(
    name=df_storage['data-Name'],
    gender=df_storage['data-Gender'],
    email=df_storage['data-e-mail'].replace({'Unspecified':pd.NA}),
    phone=df_storage['data-phone_1_-_Value'].apply(lambda x:'977'+ x if x!='null' else pd.NA),
    ct=df_storage['data-address_1_-_city'].apply(str.lower).replace({'unspecified':pd.NA}).values,
    country='NP',
    rod=pd.to_datetime(df_storage['data-today'],format="%d/%m/%y",errors='coerce'),
    dob=df_storage['data-Year_of_Birth'],
    today=df_storage['data-today'],
    photo_id=[get_pic_id(photo) for photo in df_storage['data-photo'].values],
    fn=lambda d: d['name'].apply(get_fn),
    ln=lambda d: d['name'].apply(get_ln),
    age=lambda d: d['dob'].apply(get_age),
    id=df_storage['data-meta-instanceID'],
    lisc=df_storage['data-Driving_License'].replace({
        'Unspecified':'idk', 'no':'False', 'category_a__motorcycle_scooter':'A',
        'category_a__motorcycle_scooter':'A', 'category_b__car_jeep_delivery':"B",
        'category_k__scooter__moped':"K", '':'idk', 'yes':'True',
        'category_a__motorcycle_scooter':"A",
        'category_a__motorcycle_scooter, no':"A",
        'category_b__car_jeep_delivery':"B",
        'category_b__car_jeep_delivery':"B", "category_a__motorcycle_scooter":"A",
        'no, category_a__motorcycle_scooter':"A",
        'category_b__car_jeep_delivery, category_a__motorcycle_scooter':'AB',
        'category_a__motorcycle_scooter, category_b__car_jeep_delivery':'AB',
})
)

In [38]:
df.head()

,id,email,phone,name,gender,ct,country,rod,dob,today,photo_id,fn,ln,age,lisc
0,,<NA>,9779814300021,Aaliya Saha,Female,biratnagar,NP,2019-01-01,null,01/01/19,<NA>,Aaliya,Saha,NaN,idk
1,,<NA>,<NA>,Aarif Poudel,male,<NA>,NP,2019-01-01,null,01/01/19,<NA>,Aarif,Poudel,NaN,idk
2,,<NA>,<NA>,Aaroj Sunuwar,male,<NA>,NP,2019-01-01,null,01/01/19,<NA>,Aaroj,Sunuwar,NaN,idk
3,,<NA>,9779808105714,Aashish Rai,male,lalitpur,NP,2019-01-01,null,01/01/19,<NA>,Aashish,Rai,NaN,idk
4,,<NA>,9779826402287,Aashish Rana Magar,male,<NA>,NP,2019-01-01,null,01/01/19,<NA>,Aashish,Magar,NaN,idk


In [27]:
np.array(get_age(df_storage['data-Year_of_Birth'])).shape

(2033,)

In [6]:
df_storage.columns

Index(['data-start', 'data-today', 'data-deviceid', 'data-Name',
       'data-Year_of_Birth', 'data-Gender', 'data-address_1_-_city',
       'data-address_1_-_street', 'data-Permanent_Address_District',
       'data-phone_1_-_Value', 'data-phone_2_-_Value',
       'data-Secondary_Contact_Person', 'data-phone_3_-_Value', 'data-e-mail',
       'data-Short_Introduction', 'data-audio_introduction', 'data-photo',
       'data-Driving_License', 'data-Other_License_Categories',
       'data-Possession_of_Any_Vehicle', 'data-Vehicle_Type',
       'data-Transfer_To', 'data-Reason_for_N_A', 'data-Emcee_Level',
       'data-Brand_Promoter_Level', 'data-Model_Promoter_Level',
       'data-Helper_Level', 'data-Coordinator_Level',
       'data-Flyer_Distributor_level', 'data-Event_Volunteer_level',
       'data-Front_Line_Survey_level', 'data-Model_Height', 'data-Notes',
       'data-Screened_By', 'data-reference_no', 'data-meta-instanceID'],
      dtype='object')

In [13]:
pd.to_datetime(df_storage.loc[:,['data-start']],format="")

ValueError: to assemble mappings requires at least that [year, month, day] be specified: [day,month,year] is missing

In [35]:
data_storage['data-Gender']

NameError: name 'data_storage' is not defined